In [20]:
pip install selenium webdriver_manager

Note: you may need to restart the kernel to use updated packages.


In [21]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import re
import requests 

# Setup WebDriver
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)

url = "https://store.steampowered.com/charts/mostplayed"

try:
    # Apri la pagina
    driver.get(url)

    # Aspetta che il link di Counter-Strike 2 sia presente
    WebDriverWait(driver, 20).until(
        EC.presence_of_element_located((By.XPATH, "//a[contains(@href, '/app/730/')]"))
    )

    # Salva l'HTML della pagina
    html_content = driver.page_source

finally:
    # Chiudi il browser
    driver.quit()


InvalidSessionIdException: Message: invalid session id: session deleted as the browser has closed the connection
from disconnected: not connected to DevTools
  (Session info: chrome=141.0.7390.123); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#invalidsessionidexception
Stacktrace:
	GetHandleVerifier [0x0x6afe43+66515]
	GetHandleVerifier [0x0x6afe84+66580]
	(No symbol) [0x0x49dc48]
	(No symbol) [0x0x48ce80]
	(No symbol) [0x0x4ab81b]
	(No symbol) [0x0x511c25]
	(No symbol) [0x0x52c4d9]
	(No symbol) [0x0x50afc6]
	(No symbol) [0x0x4dc2ca]
	(No symbol) [0x0x4dd154]
	GetHandleVerifier [0x0x907353+2521315]
	GetHandleVerifier [0x0x9022d3+2500707]
	GetHandleVerifier [0x0x6d7c94+229924]
	GetHandleVerifier [0x0x6c81f8+165768]
	GetHandleVerifier [0x0x6cecad+193085]
	GetHandleVerifier [0x0x6b8158+100072]
	GetHandleVerifier [0x0x6b82f0+100480]
	GetHandleVerifier [0x0x6a25aa+11066]
	BaseThreadInitThunk [0x0x760e5d49+25]
	RtlInit

In [22]:
# Estrai gli ID dei giochi
pattern = r"/app/(\d+)/"
extracted_ids = re.findall(pattern, html_content)
extracted_ids = list(set(extracted_ids))  # Converti in un set per rimuovere i duplicati, poi torna a una lista
print(extracted_ids)


['1030300', '2062430', '1172470', '3078390', '2622380', '1281930', '236390', '3405340', '2923300', '1145350', '1174180', '365670', '292030', '440', '629520', '1158310', '4000', '105600', '218620', '1091500', '1086940', '221100', '1623730', '108600', '489830', '2767030', '322170', '271590', '2507950', '2252570', '252490', '1422450', '1973530', '730', '438100', '553850', '275850', '394360', '1245620', '1366800', '427520', '251570', '367520', '2694490', '2584990', '1049590', '322330', '1203220', '261550', '1905180', '1222670', '3241660', '359550', '766570', '3240220', '3167020', '2246340', '1142710', '2807960', '2357570', '238960', '1326470', '381210', '550', '3405690', '582660', '2300320', '513710', '582010', '2399830', '346110', '3472040', '1938090', '1364780', '813780', '284160', '3037410', '892970', '1623660', '594650', '570', '227300', '2139460', '2669320', '578080', '3681810', '294100', '250900', '413150', '3419430', '1449850', '2358720', '8930', '1818450', '646570', '230410', '2890

In [23]:
import pandas as pd
import requests
import time
from datetime import datetime
import os.path
import numpy as np

# Lista degli app_id (sostituisci con i tuoi extracted_ids)


# Dizionario per memorizzare i dati di oggi
today_data = {}

# Definisci today (mancante nel codice originale)
today = pd.Timestamp.today().strftime('%Y-%m-%d')

# Definisci csv_file (mancante nel codice originale)
csv_file = "steam_players_data.csv"

# Raccogli i dati per ogni app_id
for app_id in extracted_ids:
    try:
        # Fai la richiesta all'API di Steam
        api_url = f"http://api.steampowered.com/ISteamUserStats/GetNumberOfCurrentPlayers/v1/?appid={app_id}&format=json"
        response = requests.get(api_url)
        
        if response.status_code == 200:
            data = response.json()
            player_count = data.get("response", {}).get("player_count", np.nan)
            today_data[app_id] = player_count
        else:
            print(f"Errore nella richiesta per l'ID {app_id}: {response.status_code}")
            today_data[app_id] = np.nan
            
        # Aggiungi un ritardo per evitare di sovraccaricare l'API
        time.sleep(0.5)
        
    except Exception as e:
        print(f"Errore durante l'elaborazione dell'ID {app_id}: {e}")
        today_data[app_id] = np.nan

# Crea un DataFrame con i dati di oggi - CORREZIONE: trasforma il dizionario in una riga
today_df = pd.DataFrame(today_data, index=[today]).transpose()
today_df.columns = [today]
today_df.index.name = 'AppID'

# Controlla se il file CSV esiste già
if os.path.isfile(csv_file):
    # Carica il file CSV esistente
    existing_df = pd.read_csv(csv_file, index_col='AppID')
    
    # CORREZIONE: gestione corretta dell'aggiornamento dei dati
    if today in existing_df.columns:
        print(f"I dati per {today} sono già presenti nel file. Sovrascrivendo...")
        existing_df[today] = today_df[today]
    else:
        # Aggiungi i dati di oggi al DataFrame esistente
        existing_df[today] = today_df[today]
    
    # Salva il DataFrame aggiornato
    existing_df.to_csv(csv_file)
    print(f"Dati per {today} aggiunti al file esistente.")
else:
    # Se il file non esiste, crea un nuovo file con i dati di oggi
    today_df.to_csv(csv_file)
    print(f"Nuovo file creato con i dati per {today}.")

# Visualizza i dati di oggi
print("\nDati raccolti oggi:")
print(today_df)

Errore nella richiesta per l'ID 1422450: 404
Nuovo file creato con i dati per 2025-11-04.

Dati raccolti oggi:
         2025-11-04
AppID              
1030300     19772.0
2062430     10260.0
1172470     65301.0
3078390     20455.0
2622380     12234.0
...             ...
230410      50172.0
289070      24756.0
431960      92859.0
2073620     19051.0
1808500    130110.0

[100 rows x 1 columns]


In [24]:
display(today_df)

,2025-11-04
AppID,
1030300,19772.0
2062430,10260.0
1172470,65301.0
3078390,20455.0
2622380,12234.0
...,...
230410,50172.0
289070,24756.0
431960,92859.0


In [25]:
df = today_df.T

In [26]:
print(df)

AppID       1030300  2062430  1172470  3078390  2622380  1281930   236390  \
2025-11-04  19772.0  10260.0  65301.0  20455.0  12234.0  23013.0  47231.0   

AppID       3405340  2923300  1145350  ...  1449850  2358720     8930  \
2025-11-04  19796.0  98737.0  11966.0  ...  12862.0  11519.0  10506.0   

AppID       1818450   646570   230410   289070   431960  2073620   1808500  
2025-11-04  13687.0  12743.0  50172.0  24756.0  92859.0  19051.0  130110.0  

[1 rows x 100 columns]


In [27]:
display(df)

AppID,1030300,2062430,1172470,3078390,2622380,1281930,236390,3405340,2923300,1145350,...,1449850,2358720,8930,1818450,646570,230410,289070,431960,2073620,1808500
2025-11-04,19772.0,10260.0,65301.0,20455.0,12234.0,23013.0,47231.0,19796.0,98737.0,11966.0,...,12862.0,11519.0,10506.0,13687.0,12743.0,50172.0,24756.0,92859.0,19051.0,130110.0


In [28]:
import pandas as pd

# 1. Leggi il file CSV
input_file = "steam_players_data.csv"
df = pd.read_csv(input_file, index_col=0)  # index_col=0 usa la prima colonna come indice

# 2. Trasponi il DataFrame
df_trasposto = df.T

# 3. Salva il DataFrame trasposto in un nuovo file CSV
output_file = "steam_players_data_finale.csv"
df_trasposto.to_csv(output_file)

print(f"File trasposto salvato come {output_file}")

File trasposto salvato come steam_players_data_finale.csv
